In [1]:
## Imports

# import python libraries  
import numpy as np
import pandas as pd
import sys
import os
import time
import pickle
import tensorflow as tf
import time
from importlib import reload
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import pearsonr
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras import initializers

from scripts.splitting import split
from scripts.data_wrangling import import_all
from scripts.feature_selection import fs_functional_score, fs_atlas_landmark, fs_landmark, true_phosphosite_filter
from scripts.data_preparation import data_prep
from scripts.data_selection import data_indexing
from scripts.mean_model import MeanModel
from scripts.tf_DRP_model import train_model_multi, train_model_multi_cv

tf.keras.utils.set_random_seed(42) # set random seeds for Python, NumPy, and TensorFlow

print('Imports: Done')

# Import and wrangle phosphoproteomics, proteomics and GDSC1 drug data
phos_df, prot_df, drug_df, drug_matrix, _all_cls, _all_drugs, common_ind = import_all()

## Feature Selection 

# create list of landmark phosphosites
phosphosites = fs_functional_score(phos_df,95)
#phosphosites = fs_atlas_landmark(phos_df,80)
# remove false positive phosphosites
filtered_phosphosites = true_phosphosite_filter(phosphosites)
# reduce remaining features down by feature selection
phos_df = phos_df.filter(filtered_phosphosites,axis=1)

# create list of landmark gene proteins
landmark_proteins = fs_landmark(prot_df)
# reduce remaining features down by feature selection
prot_df = prot_df.filter(landmark_proteins,axis=1)

## Data Preparation & Selection
x_drug, x_all_phos, y_series = data_prep(drug_df,phos_df,common_ind)
x_drug, x_all_prot, y_series = data_prep(drug_df,prot_df,common_ind)
drug_cl_pairs =  y_series.index

# Cancer blind train-test split
rand_seed = 42 # set random seed for train-test split
train_size = 0.75 # set train-test fraction

train_pairs, test_pairs = split(rand_seed, _all_cls, _all_drugs, drug_cl_pairs, 
                                train_size, split_type='cblind')

# index for each set of train-test pairs
xo_train_phos, xd_train, y_train, xo_test_phos, xd_test, y_test = data_indexing(train_pairs,test_pairs,x_all_phos,x_drug,y_series) 
xo_train_prot, xd_train, y_train, xo_test_prot, xd_test, y_test = data_indexing(train_pairs,test_pairs,x_all_prot,x_drug,y_series)

Imports: Done


### Neural Network 

In [2]:
## Model Building

# Learning rate schedule function
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)


## CNN using functional API
def build_model(learning_rate=1e-2, momentum=0.9, seed=42):

    # set weight initialiser
    initializer = tf.keras.initializers.GlorotUniform(seed=seed)
    
    ## Drug input
    # one-hot encoded drug data + dense layer
    y_input = layers.Input(shape=(xd_train.shape[1]))
    y = Dense(256, kernel_initializer=initializer, activation="relu")(y_input) 
    
    
    ## Phosphoproteomics branch
    # phosphoproteomics data input
    x_input_phospho = layers.Input(shape=(xo_train_phos.shape[1],1))
    # 1st convolution layer
    x_1 = layers.Conv1D(filters=8, kernel_size=4, kernel_initializer=initializer, activation='relu')(x_input_phospho) 
    x_1 = layers.BatchNormalization()(x_1)
    x_1 = layers.MaxPooling1D()(x_1)
    x_1 = layers.Dropout(0.1)(x_1)
    # 2nd convolution layer
    x_1 = layers.Conv1D(filters=16, kernel_size=4, kernel_initializer=initializer, activation='relu')(x_1)
    x_1 = layers.BatchNormalization()(x_1)
    x_1 = layers.MaxPooling1D()(x_1)
    x_1 = layers.Dropout(0.1)(x_1)
    x_1 = layers.Flatten()(x_1)

    # FC layer for xo_train
    x_1 = layers.Dense(256, kernel_initializer=initializer, activation='relu')(x_1)
    x_1 = layers.BatchNormalization()(x_1)
    x_1 = layers.Dropout(0.5)(x_1)
    x_1 = layers.Dense(256, kernel_initializer=initializer, activation='relu')(x_1)
    x_1 = layers.BatchNormalization()(x_1)
    x_1 = layers.Dropout(0.5)(x_1)
    
    
    ## Proteomics branch
    # proteomics data input
    x_input_prot = layers.Input(shape=(xo_train_prot.shape[1],1))
    # 1st convolution layer
    x_2 = layers.Conv1D(filters=8, kernel_size=4, kernel_initializer=initializer, activation='relu')(x_input_prot) 
    x_2 = layers.BatchNormalization()(x_2)
    x_2 = layers.MaxPooling1D()(x_2)
    x_2 = layers.Dropout(0.1)(x_2)
    # 2nd convolution layer
    x_2 = layers.Conv1D(filters=16, kernel_size=4, kernel_initializer=initializer, activation='relu')(x_2)
    x_2 = layers.BatchNormalization()(x_2)
    x_2 = layers.MaxPooling1D()(x_2)
    x_2 = layers.Dropout(0.1)(x_2)
    x_2 = layers.Flatten()(x_2)

    # FC layer for xo_train
    x_2 = layers.Dense(128, kernel_initializer=initializer, activation='relu')(x_2)
    x_2 = layers.BatchNormalization()(x_2)
    x_2 = layers.Dropout(0.5)(x_2)
    x_2 = layers.Dense(128, kernel_initializer=initializer, activation='relu')(x_2)
    x_2 = layers.BatchNormalization()(x_2)
    x_2 = layers.Dropout(0.5)(x_2) 
    
    # Concat
    z = layers.concatenate([x_1, x_2, y])
    z = layers.Dense(128, kernel_initializer=initializer, activation='relu')(z)
    z = layers.BatchNormalization()(z)
    z = layers.Dense(128, kernel_initializer=initializer, activation='relu')(z)
    z = layers.BatchNormalization()(z)
    z = layers.Dense(1, kernel_initializer=initializer)(z)

    model = tf.keras.Model([x_input_phospho, x_input_prot, y_input], z)
    
    model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=learning_rate, 
                                                        momentum=momentum), 
                                                        loss='mse', metrics=['mae'])
    return model

print('Model Building: Done')

Model Building: Done


In [3]:
m_func = build_model() # set model and parameters
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler) # set learning rate scheduler
epochs = 100

train_model_multi(m_func,lr_scheduler, 
          x_train = [xo_train_phos, xo_train_prot, xd_train], 
          x_test = [xo_test_phos, xo_test_prot, xd_test], 
          y_train = y_train, y_test = y_test, 
          epochs = epochs)

Epoch 1/100
324/324 [==============================] - 22s 39ms/step - loss: 0.9690 - mae: 0.6768 - val_loss: 0.5860 - val_mae: 0.6126 - lr: 0.0100
Epoch 2/100
324/324 [==============================] - 12s 37ms/step - loss: 0.4618 - mae: 0.5230 - val_loss: 0.4278 - val_mae: 0.5252 - lr: 0.0100
Epoch 3/100
324/324 [==============================] - 11s 34ms/step - loss: 0.4179 - mae: 0.4965 - val_loss: 1.4666 - val_mae: 0.9827 - lr: 0.0100
Epoch 4/100
324/324 [==============================] - 11s 34ms/step - loss: 0.4087 - mae: 0.4920 - val_loss: 0.3612 - val_mae: 0.4641 - lr: 0.0100
Epoch 5/100
324/324 [==============================] - 11s 34ms/step - loss: 0.3890 - mae: 0.4806 - val_loss: 0.8593 - val_mae: 0.8112 - lr: 0.0100
Epoch 6/100
324/324 [==============================] - 11s 33ms/step - loss: 0.4332 - mae: 0.5107 - val_loss: 14412032.0000 - val_mae: 1116.9795 - lr: 0.0100
Epoch 7/100
324/324 [==============================] - 11s 34ms/step - loss: 0.3916 - mae: 0.4839 - va

In [4]:
epochs = 100
rand_seed_list = [913,425]
model = build_model() # set model and parameters
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler) # set learning rate scheduler

train_model_multi_cv(model,lr_scheduler,train_pairs,
             x_all = [x_all_phos,x_all_prot],
             x_drug = x_drug,y_series = y_series,
             epochs=epochs,rand_seed_list=rand_seed_list)

Seed 913:

K-fold 1
Restoring model weights from the end of the best epoch: 12.
Epoch 22: early stopping
239.232 seconds
r2  score:  0.7598200840225259
mse score:  0.22530355653307996
-----
K-fold 2
Restoring model weights from the end of the best epoch: 45.
Epoch 55: early stopping
589.140 seconds
r2  score:  0.8293708851389254
mse score:  0.17516349957633576
-----
K-fold 3
Restoring model weights from the end of the best epoch: 10.
Epoch 20: early stopping
215.783 seconds
r2  score:  0.819197725492145
mse score:  0.1917555424084669
-----
r2 Scores for seed 913:
0.7598200840225259
0.8293708851389254
0.819197725492145

mean r2: 0.8028
-----

Seed 425:

K-fold 1
Restoring model weights from the end of the best epoch: 19.
Epoch 29: early stopping
309.722 seconds
r2  score:  0.8297271131627653
mse score:  0.17224190020367272
-----
K-fold 2
Restoring model weights from the end of the best epoch: 3.
Epoch 13: early stopping
137.880 seconds
r2  score:  0.8205783556686086
mse score:  0.186569